In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook
# Install tmdbsimple (only need to run once)
!pip install tqdm

In [2]:
# Load API Credentials
with open('/Users/om88920/.secret/yelp_api.json') as f:
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Pizza'

In [5]:
LOCATION.split(',')[0]

'NY'

In [6]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_NY_pizza.json"
JSON_FILE

'Data/results_in_progress_NY_pizza.json'

In [7]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_pizza.json already exists.


In [8]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 440 previous results found.


In [9]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [10]:
## How many results total?
total_results = results['total']
total_results

12400

In [11]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [12]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

598

In [13]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [14]:
from tqdm.notebook import tqdm_notebook
import time
for page_num in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2)

  0%|          | 0/598 [00:00<?, ?it/s]

In [15]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [16]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_NY_pizza.json already exists. Deleting previous file...
[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to new file.
- 0 previous results found.


625

In [17]:
for page_num in tqdm_notebook(range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 450:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/625 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [18]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/PfI8oV...,False,https://www.yelp.com/biz/prince-street-pizza-n...,5084,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[delivery, pickup]",$,"{'address1': '27 Prince St', 'address2': None,...",12129664100,(212) 966-4100,1961.877142
1,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/od36nF...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2706,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",17185966700,(718) 596-6700,308.569844
2,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media3.fl.yelpcdn.com/bphoto/l0Phrn...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,3193,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",12129650500,(212) 965-0500,1932.946770
3,v1DHGRNCH9247WLYoaoA9A,l-industrie-pizzeria-brooklyn,L'industrie Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/1-VBwA...,False,https://www.yelp.com/biz/l-industrie-pizzeria-...,1152,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.71162, 'longitude': -73.95783}",[delivery],$,"{'address1': '254 S 2nd St', 'address2': '', '...",17185990002,(718) 599-0002,3145.016041
4,uc5qQMzs96rzjK27epDCug,joes-pizza-new-york-4,Joe's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/FKjd9R...,False,https://www.yelp.com/biz/joes-pizza-new-york-4...,3073,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.73060076, 'longitude': -74.002...",[delivery],$,"{'address1': '7 Carmine St', 'address2': '', '...",12123661182,(212) 366-1182,2874.702514


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
435,xxgcU_oFIox_pAmroo9WwA,front-street-pizza-brooklyn,Front Street Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/wRdm4f...,False,https://www.yelp.com/biz/front-street-pizza-br...,472,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.70244, 'longitude': -73.98943}","[pickup, delivery]",$,"{'address1': '80 Front St', 'address2': '', 'a...",17188753700,(718) 875-3700,550.439226
436,hGksyTiHBBdrhbiVjXzWVw,scarrs-pizza-new-york,Scarr's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/xonxGJ...,False,https://www.yelp.com/biz/scarrs-pizza-new-york...,875,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.7157238, 'longitude': -73.9916...","[pickup, delivery]",$,"{'address1': '35 Orchard St', 'address2': '', ...",12123343481,(212) 334-3481,1165.100255
437,afuiZyn9oOyyCpuW7wGfHQ,l-antica-pizzeria-da-michele-new-york,L’Antica Pizzeria da Michele,https://s3-media2.fl.yelpcdn.com/bphoto/CFXMUP...,False,https://www.yelp.com/biz/l-antica-pizzeria-da-...,106,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.737052, 'longitude': -74.001571}","[pickup, delivery]",$$$,"{'address1': '81 Greenwich Ave', 'address2': N...",19295246682,(929) 524-6682,3560.290394
438,-9NO2IDCJMyNGZMAQ1-VQw,ny-pizza-suprema-new-york,NY Pizza Suprema,https://s3-media4.fl.yelpcdn.com/bphoto/EqbDXn...,False,https://www.yelp.com/biz/ny-pizza-suprema-new-...,2552,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.7502059936523, 'longitude': -7...","[pickup, delivery]",$,"{'address1': '413 8th Ave', 'address2': '', 'a...",12125948939,(212) 594-8939,4977.847939
439,j-vsTCIrppw9w7mOB_KHhg,pizza-loves-sauce-new-york-2,Pizza Loves Sauce,https://s3-media2.fl.yelpcdn.com/bphoto/OfskPM...,False,https://www.yelp.com/biz/pizza-loves-sauce-new...,25,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,"{'latitude': 40.723026, 'longitude': -73.989664}","[pickup, delivery]",NaN,"{'address1': '147 E Houston St', 'address2': N...",19174561013,(917) 456-1013,1986.675302


In [19]:
# Assuming final_df is your DataFrame
columns_with_lists = []

for column in final_df.columns:
    if any(isinstance(item, list) for item in final_df[column]):
        columns_with_lists.append(column)

print("Columns with lists:", columns_with_lists)

Columns with lists: ['categories', 'transactions']


In [20]:
final_df = final_df.explode('categories')

In [21]:
final_df = final_df.explode('transactions')

In [22]:
columns_with_dicts = []

for column in final_df.columns:
    if any(isinstance(item, dict) for item in final_df[column]):
        columns_with_dicts.append(column)

print("Columns with dictionaries:", columns_with_dicts)

Columns with dictionaries: ['categories', 'coordinates', 'location']


In [23]:
final_df = final_df.explode('coordinates')

In [24]:
final_df = final_df.explode('categories')

In [26]:
final_df = final_df.explode('location')

In [27]:
# check for duplicate results
final_df.duplicated().sum()

38048

In [28]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

39211

In [29]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [30]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)